<a href="https://colab.research.google.com/github/nanoNNsk/superaiss5_hackathon/blob/main/nlp_hackathon_superaiss5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# upload file

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# data prepare

In [11]:
!pip install pandas

In [90]:
#function for load data in folder with .txt
import os
import pandas as pd
def load_data(folder_path):
    data = []
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith(".txt"):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                for line_number, line in enumerate(file):
                    file_id = filename.split('.')[0]  # ตัดส่วน .txt ออกไป
                    data.append({'id': f'{file_id}_{line_number}', 'text': line.strip()})
    df = pd.DataFrame(data)
    split_columns = df['text'].str.split('\t', expand=True)
    result = pd.concat([df['id'], split_columns], axis=1)
    result.columns = ['sentence_id', 'word', 'pos', 'ner', 'more']

    return result

def load_data_test(folder_path):
    data = []
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith(".txt"):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                for line_number, line in enumerate(file):
                    file_id = filename.split('.')[0]  # ตัดส่วน .txt ออกไป
                    data.append({'id': f'{file_id}_{line_number}', 'text': line.strip()})
    df = pd.DataFrame(data)
    split_columns = df['text'].str.split('\t', expand=True)
    result = pd.concat([df['id'], split_columns], axis=1)
    result.columns = ['id', 'word', 'pos', 'ner']

    return result

In [91]:
# set path to dataset
train_folder = '/content/drive/MyDrive/hackathon_superaiss5/nlp/superai_ner/train/train'
eval_folder = '/content/drive/MyDrive/hackathon_superaiss5/nlp/superai_ner/eval/eval'
test_folder = '/content/drive/MyDrive/hackathon_superaiss5/nlp/superai_ner/test/test'

In [92]:
train_data = load_data(train_folder)
eval_data = load_data(eval_folder)
test_data = load_data_test(test_folder)

In [93]:
train_data.head()

,sentence_id,word,pos,ner,more
0,00001_0,สภาสังคมสงเคราะห์แห่งประเทศ,NN,B_ORG,B_CLS
1,00001_1,ไทย,NN,E_ORG,I_CLS
2,00001_2,จี้,VV,O,I_CLS
3,00001_3,ศาล,NN,O,I_CLS
4,00001_4,ไฟเขียว,VV,O,I_CLS


In [94]:
test_data.head()

,id,word,pos,ner
0,03795_0,รัฐ,NN,B_CLS
1,03795_1,ถังแตก,VV,I_CLS
2,03795_2,วิก,NN,I_CLS
3,03795_3,_,NN,I_CLS
4,03795_4,7,NN,I_CLS


In [95]:
train_data.drop(columns=['pos','more'], inplace=True)
eval_data.drop(columns=['pos','more'], inplace=True)
test_data.drop(columns=['pos','ner'], inplace=True)

In [96]:
train_data.head()

,sentence_id,word,ner
0,00001_0,สภาสังคมสงเคราะห์แห่งประเทศ,B_ORG
1,00001_1,ไทย,E_ORG
2,00001_2,จี้,O
3,00001_3,ศาล,O
4,00001_4,ไฟเขียว,O


In [97]:
eval_data.head()

,sentence_id,word,ner
0,04278_0,โฆษก,O
1,04278_1,กอส.,B_ORG
2,04278_2,ตำหนิ,O
3,04278_3,แมนฯ,B_ORG
4,04278_4,_,I_ORG


In [102]:
train_data.rename(columns={'word': 'words', 'ner': 'labels'}, inplace=True)
eval_data.rename(columns={'word': 'words', 'ner': 'labels'}, inplace=True)

In [98]:
test_data.head()

,id,word
0,03795_0,รัฐ
1,03795_1,ถังแตก
2,03795_2,วิก
3,03795_3,_
4,03795_4,7


In [38]:
labels = pd.read_csv('/content/drive/MyDrive/hackathon_superaiss5/nlp/superai_ner/tag_list.csv')
labels.head()

,tag,class
0,O,0
1,B_ORG,1
2,B_PER,2
3,B_LOC,3
4,B_MEA,4


In [84]:
label_list = labels['tag'].tolist()
label_list

['O',
 'B_ORG',
 'B_PER',
 'B_LOC',
 'B_MEA',
 'I_DTM',
 'I_ORG',
 'E_ORG',
 'I_PER',
 'B_TTL',
 'E_PER',
 'B_DES',
 'E_LOC',
 'B_DTM',
 'B_NUM',
 'I_MEA',
 'E_DTM',
 'E_MEA',
 'I_LOC',
 'I_DES',
 'E_DES',
 'I_NUM',
 'E_NUM',
 'B_TRM',
 'B_BRN',
 'I_TRM',
 'E_TRM',
 'I_TTL',
 'I_BRN',
 'E_BRN',
 'E_TTL',
 'B_NAME']

# load model

In [81]:
!pip install simpletransformers

In [82]:
from simpletransformers.ner import NERModel, NERArgs

In [99]:
# Configure the model
model_args = NERArgs()
model_args.train_batch_size = 64
model_args.num_train_epochs = 30
model_args.overwrite_output_dir = True
model_args.reprocess_input_data = True
model_args.evaluate_during_training = True

In [100]:
# Initialize the NER model
model = NERModel(
    # model_type="roberta",
    model_type="auto",
    model_name="FacebookAI/xlm-roberta-base",
    labels=label_list,
    args=model_args,
    use_cuda=True
)

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Train the model
model.train_model(train_data=train_data, eval_data=eval_data)

  0%|          | 0/3 [00:00<?, ?it/s]